# A.

In [1]:
def parse_file(hierarchy_file, id_to_name_file):
    hierarchy = {}
    id_to_name = {}

    with open(hierarchy_file, "r") as f:
        for line in f:
            parent, child = line.strip().split()
            hierarchy[child] = parent
    
    with open(id_to_name_file, "r") as f:
        for line in f:
            id, name = line.split("\t")
            id_to_name[id] = name
    return hierarchy, id_to_name

In [2]:
def create_data_structure(hierarchy):
    data_structure = {}

    for child, parent in hierarchy.items():
        if parent not in data_structure:
            data_structure[parent] = {'parent': None, 'siblings': [], 'children': [child]}
        else:
            data_structure[parent]['children'].append(child)
    
        if child not in data_structure:
            data_structure[child] = {'parent': parent, 'siblings': [], 'children': []}
        else:
            data_structure[child]['parent'] = parent
        
        for sibling in data_structure[parent]['children']:
            if sibling != child:
                data_structure[child]['siblings'].append(sibling)

    return data_structure

In [5]:
class ClassHierarchy:
    def __init__(self, data_structure):
        self.data_structure = data_structure

    def get_siblings(self, class_name):
        return self.data_structure[class_name]['siblings']

    def get_parent(self, class_name):
        return self.data_structure[class_name]['parent']

    def get_ancestors(self, class_name):
        ancestors = []
        parent = self.get_parent(class_name)
        while parent is not None:
            ancestors.append(parent)
            parent = self.get_parent(parent)
        return ancestors

    def is_same_ancestor(self, class1, class2):
        return bool(set(self.get_ancestors(class1)) & set(self.get_ancestors(class2)))


In [7]:
hierarchy, id_to_name = parse_file('hierarchy.txt', 'id_to_name.txt')
data_structure = create_data_structure(hierarchy)
class_hierarchy = ClassHierarchy(data_structure)


In [10]:
class_name = 'n02118333'

print(f"Siblings of {class_name}: {class_hierarchy.get_siblings(class_name)}")

print(f"Parent of {class_name}: {class_hierarchy.get_parent(class_name)}")

print(f"Ancestors of {class_name}: {class_hierarchy.get_ancestors(class_name)}")

Siblings of n02118333: ['n02115335', 'n02117135', 'n02083672', 'n02115096']
Parent of n02118333: n02083346
Ancestors of n02118333: ['n02083346', 'n02075296', 'n01886756', 'n01861778', 'n01471682', 'n01466257', 'n00015388', 'n00004475', 'n00004258', 'n00003553', 'n00002684', 'n00001930', 'n00001740']


# B.
1. Binary CrossEntropy Loss
2. Hierarchical Softmax Loss
3. Focal Loss

# C.

In [13]:
# NMS base
import numpy as np

def non_max_suppression(boxes, scores, threshold):
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        
        inds = np.where(ovr <= threshold)[0]
        order = order[inds + 1]
    
    return keep



In [14]:
# 1. Select boxes with confidence threshold that exceed a threshold
# 2. For highly overlapped boxes that belong to the same class, eliminate the one with
# lower confidence
# 3. For highly overlapped boxes that belong to different classes, eliminate the one that is
# the parent class of another box. For instance, box1 (Apple) and box2 (Fruit) are
# highly overlapped. Box 2 should be eliminated since box2 is a parent class of box1
# 4. The NMS function should accept arguments:
# i) bounding box proposals
# ii) scores/confidence
# iii) class
# iv) class hierarchy
# v) score threshold
# vi) iou threshold

def non_max_suppression(boxes, scores, classes, class_hierarchy, threshold, confidence_threshold, iou_threshold):

    # Dựa trên mức threshold. Nó tìm trên mức threshold exceed là "confidence_threshold"
    indices = np.where(scores >= confidence_threshold)[0]
    boxes = boxes[indices]
    scores = scores[indices]
    classes = classes[indices]
    
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(indices[i])
        
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        # tính toán toạ độ của khu vực giao nhau giữa box(i) với box còn lại
        
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        # lấy max
        
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)
        #tính toán IOU giữa box chọn vs box còn lại
        
        diff_class_indices = np.where(classes[order[1:]] != classes[i])[0]
        diff_class_order = order[diff_class_indices + 1]
        diff_class_iou = iou[diff_class_indices]
        
        if len(diff_class_order) > 0:
            parent_classes = [class_hierarchy.get(classes[c], classes[c]) for c in diff_class_order]
            parent_indices = np.where(np.isin(classes[order[1:]], parent_classes))[0]
            parent_order = order[parent_indices + 1]
            parent_iou = iou[parent_indices]
            
            eliminate = np.logical_and(diff_class_iou > iou_threshold, parent_iou > iou_threshold)
            eliminate_indices = diff_class_order[eliminate]
            
            order = np.delete(order, eliminate_indices)
            iou = np.delete(iou, eliminate_indices)

        #loại bỏ các box chồng chéo và thuộc các layer khác nhau. 
        # kiểm tra xem có bất kì biox nào thuộc các lớp khác ko. có thì tạo 1 list lớp parent bằng class_hierarchy.
        # tìm các box còn lại ('order[1:]') có lớp cha trùng với lớp cha trong 'parent_class'
        # cập nhật order vs iou
        
        inds = np.where(iou <= threshold)[0]
        order = order[inds + 1]
        #chỉ các box có giá trị IOU dưới ngưỡng threshold cập nhật mảng order
    
    return keep